## Better Together: Unleash AI-powered BI with Snowflake + Quick Sight 
Refer to the [Quickstart](add github/sfquicks_start link) demostrates the integration between Snowflake and Amazon Quick Sight, showcasing Snowflake's semantic view, a new schema level objects. 


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- ======================================================
-- PART 1: Snowflake Setup for semantic view quick start
-- To get started, we will CREATE a database called MOVIES and schema PUBLIC 
-- warehouse to auto-suspends after 30 mins (1800 seconds)
-- ======================================================

USE ROLE ACCOUNTADMIN;

-- Create role for semantic view quick start
CREATE ROLE IF NOT EXISTS semantic_quick_start_role
COMMENT = 'Role for semantic view quick start demo';

-- Set variables
SET
    my_user = CURRENT_USER();
    
--Grant role to your user 
    GRANT ROLE semantic_quick_start_role TO USER IDENTIFIER($my_user);
    
-- create database, schema and warehouse
    CREATE WAREHOUSE IF NOT EXISTS WORKSHOPWH WITH WAREHOUSE_SIZE = 'XSMALL' AUTO_SUSPEND = 1800 AUTO_RESUME = TRUE COMMENT = 'Warehouse for semantic view quick start demo';
CREATE DATABASE IF NOT EXISTS movies;
GRANT OWNERSHIP ON DATABASE movies TO ROLE semantic_quick_start_role COPY CURRENT GRANTS;
GRANT OWNERSHIP ON SCHEMA movies.PUBLIC TO ROLE semantic_quick_start_role COPY CURRENT GRANTS;
GRANT OWNERSHIP ON WAREHOUSE workshopwh TO ROLE semantic_quick_start_role COPY CURRENT GRANTS;
-- Grant privileges to create semantic views
GRANT CREATE SEMANTIC VIEW ON SCHEMA movies.PUBLIC TO ROLE semantic_quick_start_role;
GRANT CREATE STAGE ON SCHEMA movies.PUBLIC TO ROLE semantic_quick_start_role;

In [ ]:
-- =============================================
-- PART 1: Snowflake Setup for semantic view quick start
-- =============================================

-- Verify the below information
SELECT
  CURRENT_DATABASE() AS current_db,
  CURRENT_SCHEMA()   AS current_schema,
  CURRENT_ROLE()     AS current_role,
  CURRENT_USER() AS current_user;


In [ ]:
-- =============================================
-- PART 1: Snowflake Setup for semantic view quick start
-- =============================================

-- Set variables for the specified role, database, and schema
SET my_role = 'semantic_quick_start_role';
SET my_db = CURRENT_DATABASE();
SET my_schema = CURRENT_SCHEMA();
SET my_full_schema = $my_db || '.' || $my_schema;

In [ ]:
USE ROLE semantic_quick_start_role;
USE DATABASE movies;

In [ ]:
-- Verify the below information
SELECT
  CURRENT_DATABASE() AS current_db,
  CURRENT_SCHEMA()   AS current_schema,
  CURRENT_ROLE()     AS current_role,
  CURRENT_USER() AS current_user;

In [ ]:
-- ==============================================
-- PART 2: DATA SETUP (as semantic_quick_start_role)
-- ==============================================
-- Create Tables

-- create demo table for our movie data.
CREATE OR REPLACE TABLE movies_raw  (
    movieid int,
    title varchar,
    genres varchar
);

-- create demo table for our movie rating data.
CREATE OR REPLACE TABLE ratings_raw (
    userid int,
    movieid int,
    rating float,
    timestamp timestamp_ntz,
    firstname varchar,
    lastname varchar,
    street varchar,
    city varchar,
    state varchar,
    postcode varchar,
    country varchar,
    email varchar,
    phonenumber varchar
);

-- create curated tables for our processed data.
CREATE TABLE movies_curated
(
  movieid number,
  title varchar,
  release integer
);

CREATE TABLE genres_curated
(
  genresid number autoincrement start 1 increment 1,
  genres varchar
);

CREATE TABLE movies_genres_curated
(
  genresid number,
  movieid number
);

CREATE TABLE ratings_curated
(
    userid int,
    movieid int,
    rating float,
    timestamp timestamp_ntz
);

CREATE TABLE users_curated
(
    userid int,
    firstname varchar,
    lastname varchar,
    street varchar,
    city varchar,
    state varchar,
    postcode varchar,
    country varchar,
    email varchar,
    phonenumber varchar
);

In [ ]:
-- ==============================================
-- PART 2: DATA SETUP (as semantic_quick_start_role)
-- ==============================================
-- Load data into table

--
-- Scale up the warehouse to load the data
--
ALTER WAREHOUSE workshopwh SET WAREHOUSE_SIZE = LARGE;

CREATE OR REPLACE FILE FORMAT movielens_ffmt
  TYPE = CSV
  FIELD_DELIMITER = ','
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  COMPRESSION = gzip;

-- movies data
COPY INTO movies_raw
  FROM s3://jhs-sf-aws-bucket/movies.csv.gz
  FILE_FORMAT = (FORMAT_NAME = 'movielens_ffmt');

SELECT * FROM movies_raw LIMIT 10;

In [ ]:
SELECT count(*) FROM movies_raw;

In [ ]:
-- ==============================================
-- PART 2: DATA SETUP (as semantic_quick_start_role) 
-- ==============================================
-- Load data into table

-- rating data 
COPY INTO ratings_raw
FROM s3://jhs-sf-aws-bucket/ratings.csv.gz 
FILE_FORMAT=(FORMAT_NAME = 'movielens_ffmt');
    
-- Scale back down the warehouse
ALTER WAREHOUSE workshopwh SET WAREHOUSE_SIZE = SMALL;

SELECT * FROM ratings_raw LIMIT 10;

In [ ]:
-- verify 309,662 records loaded 
select
    count(*)
from
    ratings_raw;

In [ ]:
-- Load raw ratings data into ratings_curated table.

-- ratings_curated
INSERT INTO ratings_curated (userid, movieid, rating, timestamp)
SELECT userid, movieid, rating, timestamp
FROM ratings_raw;

In [ ]:
-- users_curated
INSERT INTO users_curated(userid, firstname, lastname, street, city, state, postcode, country, email, phonenumber)
SELECT DISTINCT(userid), firstname, lastname, street, city, state, postcode, country, email, phonenumber
FROM ratings_raw
GROUP BY ALL;

In [ ]:
-- Insert the curated movied data
INSERT INTO  movies_curated
SELECT movieid,  substr(title,0,regexp_instr(title, '\([0-9]{4}\)')-2) as title,
    regexp_substr(title, '([0-9]{4})') as myear
FROM movies_raw
WHERE myear is not null;

In [ ]:
SELECT * FROM ratings_curated LIMIT 10;

In [ ]:
SELECT * FROM users_curated LIMIT 10;

In [ ]:
SELECT * FROM movies_curated LIMIT 10;

In [ ]:
-- Define the Semantic View
CREATE OR REPLACE SEMANTIC VIEW MOVIE_ANALYTICS_SV
TABLES (
  MOVIES AS MOVIES.PUBLIC.MOVIES_CURATED
    PRIMARY KEY (MOVIEID)
    WITH SYNONYMS ('movies', 'films')
    COMMENT = 'Movie dimension',

  USERS AS MOVIES.PUBLIC.USERS_CURATED
    PRIMARY KEY (USERID)
    WITH SYNONYMS ('users', 'audience', 'viewers')
    COMMENT = 'User dimension',

  RATINGS AS MOVIES.PUBLIC.RATINGS_CURATED
    PRIMARY KEY (USERID, MOVIEID)
    WITH SYNONYMS ('ratings', 'reviews')
    COMMENT = 'Movie ratings fact'
)

RELATIONSHIPS (
  RATINGS_TO_MOVIES AS
    RATINGS(MOVIEID) REFERENCES MOVIES,

  RATINGS_TO_USERS AS
    RATINGS(USERID) REFERENCES USERS
)

FACTS (
  RATINGS.rating_value AS RATINGS.RATING
)

DIMENSIONS (
  -- Movie dimensions
  MOVIES.movie_title AS MOVIES.TITLE
    WITH SYNONYMS = ('film title', 'movie name')
    COMMENT = 'Title of the movie',
  MOVIES.release_year AS MOVIES.RELEASE
    COMMENT = 'Year the movie was released',

  -- User dimensions
  USERS.user_full_name AS CONCAT(USERS.FIRSTNAME, ' ', USERS.LASTNAME)
    WITH SYNONYMS = ('audience name', 'viewer name')
    COMMENT = 'Full name of the user',
  USERS.user_first_name AS USERS.FIRSTNAME
    COMMENT = 'First name of the user',
  USERS.user_email AS USERS.EMAIL
    WITH SYNONYMS = ('email address', 'contact email')
    COMMENT = 'Email address of the user',
  USERS.user_last_name AS USERS.LASTNAME
    COMMENT = 'Last name of the user',
  USERS.user_city AS USERS.CITY
    COMMENT = 'City where user is located',
  USERS.user_state AS USERS.STATE
    COMMENT = 'State where user is located',
  USERS.user_country AS USERS.COUNTRY
    COMMENT = 'Country where user is located',

  -- ID dimensions
  USERS.user_id AS USERS.USERID
    COMMENT = 'User ID',
  MOVIES.movie_id AS MOVIES.MOVIEID
    COMMENT = 'Movie ID',

  -- Rating timestamp dimensions
  RATINGS.rating_timestamp AS RATINGS.TIMESTAMP
    WITH SYNONYMS = ('review date', 'rating date')
    COMMENT = 'When the rating was given',
  RATINGS.rating_year AS YEAR(RATINGS.TIMESTAMP)
    WITH SYNONYMS = ('review year', 'year rated')
    COMMENT = 'Year when the rating was given',
  RATINGS.rating_month AS MONTH(RATINGS.TIMESTAMP)
    WITH SYNONYMS = ('review month', 'month rated')
    COMMENT = 'Month when the rating was given',
  RATINGS.rating_day AS DAY(RATINGS.TIMESTAMP)
    WITH SYNONYMS = ('review day', 'day rated')
    COMMENT = 'Day when the rating was given',

  -- Rating ID dimensions
  RATINGS.user_id AS RATINGS.USERID
    COMMENT = 'User ID from ratings table',
  RATINGS.movie_id AS RATINGS.MOVIEID
    COMMENT = 'Movie ID from ratings table'
)

METRICS (
  RATINGS.total_ratings AS COUNT(RATINGS.rating_value)
    WITH SYNONYMS = ('total reviews', 'rating count', 'number of ratings')
    COMMENT = 'Count of all rating values in the ratings table',

  RATINGS.avg_rating AS AVG(RATINGS.rating_value)
    WITH SYNONYMS = ('average rating', 'mean score', 'rating average')
    COMMENT = 'Average of all rating values in the ratings table',

  USERS.distinct_users AS COUNT(DISTINCT USERS.user_id)
    WITH SYNONYMS = ('total users', 'user count', 'number of users')
    COMMENT = 'Count of distinct user IDs from the users table',

  MOVIES.distinct_movies AS COUNT(DISTINCT MOVIES.movie_id)
    WITH SYNONYMS = ('total movies', 'movie count', 'number of movies')
    COMMENT = 'Count of distinct movie IDs from the movies table',

  RATINGS.distinct_users AS COUNT(DISTINCT RATINGS.user_id)
    WITH SYNONYMS = ('active users', 'users who rated', 'rating participants')
    COMMENT = 'Count of distinct user IDs from the ratings table',

  RATINGS.distinct_movies AS COUNT(DISTINCT RATINGS.movie_id)
    WITH SYNONYMS = ('rated movies', 'movies with ratings', 'reviewed films')
    COMMENT = 'Count of distinct movie IDs from the ratings table',

  RATINGS.popularity_score AS RATINGS.total_ratings * RATINGS.avg_rating
    WITH SYNONYMS = ('movie popularity', 'engagement score', 'quality index')
    COMMENT = 'Popularity score combining volume and quality'
)

--FILTER (
 -- recent_ratings AS RATINGS.rating_timestamp >= DATEADD(year, -1, CURRENT_TIMESTAMP())
 --   COMMENT = 'Ratings submitted in the last year',

 -- high_ratings AS RATINGS.rating_value >= 4
 --   COMMENT = 'Ratings with score 4 or higher'
--)

COMMENT = 'Semantic view for movie ratings and user behavior analytics, including time-grain dimensions and popularity score';

In [ ]:
SHOW semantic views;

In [ ]:
-- 1. Top movies by release year
SELECT * FROM SEMANTIC_VIEW (
  MOVIE_ANALYTICS_SV
  DIMENSIONS MOVIES.release_year, MOVIES.movie_title
  METRICS RATINGS.total_ratings, RATINGS.avg_rating
)
WHERE release_year >= 2020 AND release_year <=2026
ORDER BY total_ratings DESC;

In [ ]:
-- 2. Highly rated movies released after 2015
SELECT * FROM SEMANTIC_VIEW (
  MOVIE_ANALYTICS_SV
  DIMENSIONS MOVIES.movie_title, MOVIES.release_year
  METRICS RATINGS.avg_rating, RATINGS.total_ratings
)
WHERE release_year > 2015 
  AND avg_rating >= 4.0
ORDER BY avg_rating DESC;

In [ ]:
-- 3. Countries with the highest average ratings
SELECT * FROM SEMANTIC_VIEW (
  MOVIE_ANALYTICS_SV
  DIMENSIONS USERS.user_country
  METRICS RATINGS.avg_rating, RATINGS.total_ratings
)
ORDER BY avg_rating DESC;

In [ ]:
-- Describe semantic view and save to dataframe

DESCRIBE SEMANTIC VIEW MOVIE_ANALYTICS_SV

In [ ]:
# Convert semantic view description to pandas dataframe
from snowflake.snowpark import Session
sv_metadata_df = get_active_session().sql("DESCRIBE SEMANTIC VIEW MOVIE_ANALYTICS_SV").to_pandas()
print(f"Semantic view metadata shape: {sv_metadata_df.shape}")
print("\nFirst 5 rows:")
print(sv_metadata_df.head())

# Save to CSV if needed
# sv_metadata_df.to_csv('movie_analytics_sv_metadata.csv', index=False)



In [ ]:
SELECT TO_VARCHAR( GET_DDL(
  'SEMANTIC_VIEW',
  'MOVIES.PUBLIC.MOVIE_ANALYTICS_SV'
));


## Click download button ⬇️ (top right corner of the output of the above cell) to 💾 DOWNLOAD (the DDL) AS CSV file. 

In [ ]:
from snowflake.snowpark import Session
sv_ddl_df = get_active_session().sql("SELECT TO_VARCHAR(GET_DDL('SEMANTIC_VIEW','MOVIES.PUBLIC.MOVIE_ANALYTICS_SV'))").to_pandas()

print(sv_ddl_df.head())

# Save to CSV if needed
sv_ddl_df.to_csv('movie_analytics_sv_ddl.csv', index=False)






This completes the Snowflake's object creation and data load, semantic view.

We will move on to the AWS side configuration

# QuickSight Dataset Generator - Complete End-to-End Guide

## Overview
This guide walks you through the complete process of creating QuickSight datasets from Snowflake DDL, from setting up credentials to sharing datasets with users.

## Complete Workflow

```
0. Open AWS console --> cloudshell --> Upload the zip
   ↓
1. Create AWS Secret (Snowflake credentials)
   ↓
2. Create QuickSight Data Source (using secret)
   ↓
3. Generate QuickSight Schema (from Snowflake DDL)
   ↓
4. Create Dataset (from generated schema)
   ↓
5. Start SPICE Ingestion (load data)
   ↓
6. Share Dataset (with users)
  ↓
7. From Quick Suite console, create Q topic or Dashboard
```

---

## Prerequisites

### Required Software
- Python 3.x with boto3 installed
- AWS CLI configured
- Access to Snowflake account

### Required Permissions
- AWS Secrets Manager: Create/Update secrets
- QuickSight: Create data sources and datasets
- IAM: Get caller identity

### Required Information
- Snowflake account identifier
- Snowflake database name
- Snowflake warehouse name
- Snowflake username and password
- QuickSight user to share with

---
## Step 0: Prepare the environment

1. Open AWS console

2. Click the top menu button to open cloudshell

3. Upload the zip 

4. Unzip the zip 
```bash
unzip Solution_Package.zip
```

5. Go to the folder Solution_Package
```bash
cd Solution_Package
```

6. Upload the DDL.csv downloaded from last step

---

## Step 1: Create AWS Secret for Snowflake Credentials

Store your Snowflake credentials securely in AWS Secrets Manager.

### Option A: Using Python Script 

```bash
python create_secret.py \
  --secret-name snowflake-credentials \
  --account YOUR_SNOWFLAKE_ACCOUNT \
  --database MOVIES \
  --warehouse WORKSHOPWH \
  --user YOUR_USERNAME \
  --password YOUR_PASSWORD \
  --region us-east-1 \
```

### Option B: Using Interactive Shell Script (Recommended)

```bash
./setup_secrets.sh
```

The script will prompt you for:
- Snowflake account identifier
- Database name (default: MOVIES)
- Warehouse name (default: WORKSHOPWH)
- Username
- Password
- AWS region (default: us-east-1)
- Secret name (default: snowflake-credentials)

### Verify Secret Creation

```bash
aws secretsmanager get-secret-value \
  --secret-id snowflake-credentials \
  --region us-east-1 \
```

### Expected Secret Format

```json
{
  "account": "your-account.us-east-1",
  "database": "MOVIES",
  "warehouse": "WORKSHOPWH",
  "user": "your_username",
  "password": "your_password"
}
```

---

## Step 2: Create QuickSight Data Source

Create a QuickSight data source that connects to Snowflake using the credentials from Secrets Manager.

### Command

```bash
python create_snowflake_datasource.py \
  --datasource-id movies-snowflake-datasource \
  --datasource-name "Movies Snowflake Data Source" \
  --secret-name snowflake-credentials \
  --region us-east-1 \
```

### Parameters

| Parameter | Required | Default | Description |
|-----------|----------|---------|-------------|
| --datasource-id | No | movies-snowflake-datasource | QuickSight data source ID |
| --datasource-name | No | Movies Snowflake Data Source | Display name |
| --secret-name | Yes | - | AWS Secrets Manager secret name |
| --region | No | us-east-1 | AWS region |
| --profile | No | default | AWS profile name |
| --no-delete | No | False | Keep existing data source |

### Expected Output

```
Using AWS Account ID: your-account
Region: us-east-1

Retrieving Snowflake credentials from secret: snowflake-credentials
  ✓ Snowflake Account: your-account.us-east-1
  ✓ Database: MOVIES
  ✓ Warehouse: WORKSHOPWH
  ✓ User: your_username

Creating Snowflake data source: Movies Snowflake Data Source

============================================================
✓ Data Source Created Successfully!
============================================================

Data Source Details:
  ID: movies-snowflake-datasource
  ARN: arn:aws:quicksight:us-east-1:your-account:datasource/movies-snowflake-datasource
  Status: 200
  Creation Status: CREATION_SUCCESSFUL
```

### Verify Data Source

```bash
aws quicksight describe-data-source \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-source-id movies-snowflake-datasource \
  --region us-east-1 \
```

---

## Step 3: Generate QuickSight Schema from Snowflake DDL

Convert your Snowflake Semantic View DDL (exported to CSV) into a complete QuickSight dataset schema.

### Command

```bash
python generate_quicksight_schema_v2.py \
  --csv-path SF_DDL.csv \
  --datasource-arn "arn:aws:quicksight:us-east-1:ACCOUNT_ID:datasource/movies-snowflake-datasource" \
  --database MOVIES \
  --dataset-id movie-analytics-dataset \
  --dataset-name "Movie Analytics Dataset" \
  --output quicksight_schema_complete.json
```

### Parameters

| Parameter | Required | Default | Description |
|-----------|----------|---------|-------------|
| --csv-path | No | ../SF_DDL.csv | Path to Snowflake DDL CSV file |
| --datasource-arn | Yes | - | QuickSight data source ARN |
| --database | No | MOVIES | Snowflake database name |
| --dataset-id | No | movie-analytics-dataset | QuickSight dataset ID |
| --dataset-name | No | Movie Analytics Dataset | Dataset display name |
| --output | No | quicksight_schema_complete.json | Output file path |

### Expected Output

```
Parsing Snowflake DDL from: SF_DDL.csv
Found 3 tables
Found 2 relationships
Found 17 dimensions
Found 1 facts
Found 7 metrics

Generating complete QuickSight dataset schema...

Complete schema saved to: quicksight_schema_complete.json

This schema includes:
  ✓ All 3 physical tables (MOVIES, USERS, RATINGS)
  ✓ Logical tables with joins
  ✓ Column renames based on DDL aliases
  ✓ Type casts (IDs to STRING)
  ✓ Calculated fields (USER_FULL_NAME, 7 metrics)
  ✓ Column descriptions from DDL comments
```

### What Gets Generated

The schema includes:

**Physical Tables (3)**
- RATINGS_CURATED (fact table)
- MOVIES_CURATED (dimension table)
- USERS_CURATED (dimension table)

**Transformations**
- Column renames (e.g., TITLE → MOVIE_TITLE)
- Type casts (IDs to STRING)
- Table joins (RATINGS → MOVIES → USERS)
- USER_ prefix on all user columns

**Calculated Fields (8)**
1. USER_FULL_NAME
2. MOVIES_DISTINCT_MOVIES
3. USERS_DISTINCT_USERS
4. RATINGS_TOTAL_RATINGS
5. RATINGS_AVG_RATING
6. RATINGS_DISTINCT_USERS
7. RATINGS_DISTINCT_MOVIES
8. RATINGS_POPULARITY_SCORE

---

## Step 4: Create Dataset and Start Ingestion

Create the QuickSight dataset from the generated schema and start loading data into SPICE.

### Option A: Create Dataset Without Sharing

```bash
python test_complete_schema.py \
  --region us-east-1
```

### Option B: Create Dataset and Share with User (Recommended)

```bash
python test_complete_schema.py \
  --region us-east-1 \
  --share-with "user-name"
```

### Parameters

| Parameter | Required | Default | Description |
|-----------|----------|---------|-------------|
| --profile | No | default | AWS profile name |
| --region | No | us-east-1 | AWS region |
| --share-with | No | - | Username to share with (format: namespace/username) |

### Expected Output

```
Account: your-account
Dataset ID: movie-analytics-dataset
No existing dataset
✓ Dataset created: movie-analytics-dataset
✓ Status: 201
✓ Ingestion started: ingestion-id
✓ Dataset shared with: user-name
  User ARN: arn:aws:quicksight:us-east-1:your-account:user/default/user-name
```

---

## Step 5: Monitor SPICE Ingestion

Check the status of data loading into SPICE.

### Check Ingestion Status

```bash
aws quicksight describe-ingestion \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-set-id movie-analytics-dataset \
  --ingestion-id INGESTION_ID \
  --region us-east-1 \
```

### Expected Statuses

- `INITIALIZED` - Ingestion is starting
- `QUEUED` - Waiting to start
- `RUNNING` - Currently loading data
- `COMPLETED` - Successfully loaded
- `FAILED` - Error occurred

### Successful Ingestion Output

```json
{
  "IngestionStatus": "COMPLETED",
  "RowInfo": {
    "RowsIngested": 378436,
    "RowsDropped": 0,
    "TotalRowsInDataset": 378436
  },
  "ErrorInfo": {}
}
```

---

## Step 6: Verify Dataset and Permissions

### Check Dataset Details

```bash
aws quicksight describe-data-set \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-set-id movie-analytics-dataset \
  --region us-east-1 \
```

### Check Dataset Permissions

```bash
aws quicksight describe-data-set-permissions \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-set-id movie-analytics-dataset \
  --region us-east-1 \
```

### Verify Output Columns

```bash
aws quicksight describe-data-set \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-set-id movie-analytics-dataset \
  --region us-east-1 \
  --query 'DataSet.OutputColumns[].Name'
```

Expected: 23 columns including all user columns with USER_ prefix and 7 metric calculated fields.

---

## Dataset Permissions Granted

When sharing a dataset with `--share-with`, the following **full permissions** are granted:

| Permission | Description |
|------------|-------------|
| UpdateDataSetPermissions | Manage who can access the dataset |
| DescribeDataSet | View dataset details and schema |
| DescribeDataSetPermissions | View current permissions |
| PassDataSet | Use dataset in analyses and dashboards |
| DescribeIngestion | View ingestion status and details |
| ListIngestions | List all data refresh operations |
| UpdateDataSet | Modify dataset configuration |
| DeleteDataSet | Delete the dataset |
| CreateIngestion | Start new data refresh |
| CancelIngestion | Cancel running data refresh |

**Total: 10 permissions** - Full control over the dataset

These permissions give the user complete control over the dataset including the ability to modify, delete, and manage permissions.

---

---

## [Optional] Finding QuickSight Users to Share With

### List All Namespaces

```bash
aws quicksight list-namespaces \
  --aws-account-id YOUR_ACCOUNT_ID \
  --region us-east-1 \
```

### List Users in Default Namespace

```bash
aws quicksight list-users \
  --aws-account-id YOUR_ACCOUNT_ID \
  --namespace default \
  --region us-east-1 \
  --query 'UserList[].UserName'
```

### Username Formats

| Format | Example | Use Case |
|--------|---------|----------|
| Simple | `reader1` | Simple QuickSight users |
| Federated | `quicksight-fed-us-users/email@domain.com` | Federated identity users |

---

## Troubleshooting

### Secret Creation Issues

**Error: Access Denied**
- Check IAM permissions for Secrets Manager
- Ensure you have `secretsmanager:CreateSecret` and `secretsmanager:UpdateSecret` permissions

**Error: Secret already exists**
- Use `--no-delete` flag or delete the existing secret first
- Or update the existing secret using the same command

### Data Source Creation Issues

**Error: Invalid credentials**
- Verify Snowflake credentials in the secret
- Test Snowflake connection manually
- Check Snowflake account identifier format

**Error: Warehouse not found**
- Verify warehouse name in Snowflake
- Ensure warehouse is running
- Check user has access to the warehouse

### Schema Generation Issues

**Error: CSV file not found**
- Check the path to SF_DDL.csv
- Ensure the file exists and is readable

**Error: No tables found**
- Verify the DDL CSV format
- Check that the DDL contains table definitions

### Dataset Creation Issues

**Error: Data source not found**
- Verify the data source ARN is correct
- Check that the data source was created successfully
- Ensure you're using the correct AWS account and region

**Error: Field does not exist**
- This means a calculated field references a column not in ProjectedColumns
- Check the generated schema for column names
- Verify all referenced columns exist after transformations

### Ingestion Issues

**Status: FAILED**
- Check Snowflake connection credentials
- Verify data source permissions
- Review ingestion error details using `describe-ingestion`
- Check Snowflake query history for errors

**Status: QUEUED for long time**
- Check SPICE capacity limits
- Verify no other ingestions are running
- Check QuickSight service status

### Sharing Issues

**Error: Namespace not found**
- Use `default` namespace for most users
- Check available namespaces with `list-namespaces`

**Error: User not found**
- Verify username with `list-users`
- Check the username format matches exactly
- Ensure user exists in the correct namespace

---

## Next Steps After Dataset Creation

### 1. Build Analyses
Use QuickSight Analysis to create visualizations:
- Create charts and graphs
- Add filters and parameters
- Build interactive dashboards

### 2. Create Dashboards
Publish analyses as dashboards:
- Share with broader audience
- Set up email reports
- Configure refresh schedules

### 3. Schedule Data Refreshes
Set up automatic SPICE refresh:
```bash
aws quicksight create-ingestion \
  --aws-account-id YOUR_ACCOUNT_ID \
  --data-set-id movie-analytics-dataset-v2 \
  --ingestion-id "scheduled-$(date +%s)" \
  --ingestion-type FULL_REFRESH
```

### 4. Monitor Usage
Track dataset usage and performance:
- Review SPICE capacity consumption
- Monitor query performance
- Check user access patterns

---

## Additional Resources

### AWS Documentation
- [QuickSight User Guide](https://docs.aws.amazon.com/quicksight/)
- [Secrets Manager User Guide](https://docs.aws.amazon.com/secretsmanager/)
- [Boto3 QuickSight Reference](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/quicksight.html)

### Snowflake Documentation
- [Snowflake Semantic Views](https://docs.snowflake.com/en/user-guide/semantic-views)
- [Snowflake DDL Reference](https://docs.snowflake.com/en/sql-reference/ddl-database)

### Script Files
- `create_secret.py` - Create AWS Secrets Manager secret
- `create_snowflake_datasource.py` - Create QuickSight data source
- `generate_quicksight_schema_v2.py` - Generate dataset schema
- `test_complete_schema.py` - Create dataset and share
- `setup_secrets.sh` - Interactive secret creation

---

## Quick Reference Commands

```bash
# Create secret
python create_secret.py --secret-name NAME --account ACCOUNT --user USER --password PASS

# Create data source
python create_snowflake_datasource.py --secret-name NAME

# Generate schema
python generate_quicksight_schema_v2.py --csv-path DDL.csv --datasource-arn ARN

# Create and share dataset
python test_complete_schema.py --share-with "Administrator/username"

# Check ingestion
aws quicksight describe-ingestion --aws-account-id ID --data-set-id DATASET --ingestion-id ING

# List users
aws quicksight list-users --aws-account-id ID --namespace default
```
